In [1]:
import sys

import os
import argparse
import pandas as pd

In [2]:
def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2021,type=int,help='The random seed')
    argparser.add_argument('--mode',default='vx',type=str,help='The choice of v/x/vx/xx')
    argparser.add_argument('--ood',default=3.0,type=float,help='The train dataset of OOD')
    argparser.add_argument('--ood_test',default=3.0,type=float,help='The train dataset of OOD')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=1,type=int,help='The use of GPU')
    argparser.add_argument('--des_str',default='/_/',type=str,help='The description of this running')
    argparser.add_argument('--oodtestall',default=0,type=int,help='The random seed')
    argparser.add_argument('--iter',default=3000,type=int,help='The num of iterations')
    argparser.add_argument('--version',default=1,type=int,help='The version')
    # About data setting ~~~~
    argparser.add_argument('--num',default=10000,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--num_reps',default=100,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--ate',default=0,type=float,help='The ate of constant')
    argparser.add_argument('--sc',default=1,type=float,help='The sc')
    argparser.add_argument('--sh',default=0,type=float,help='The sh')
    argparser.add_argument('--one',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--depX',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--depU',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--VX',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mV',default=2,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mX',default=10,type=int,help='The dim of Confounding variables X')
    argparser.add_argument('--mU',default=4,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--mXs',default=2,type=int,help='The dim of Noise variables X')
    argparser.add_argument('--storage_path',default='../../Data/',type=str,help='The dir of data storage')
    # Syn
    argparser.add_argument('--syn_alpha',default=0.01,type=float,help='')
    argparser.add_argument('--syn_lambda',default=0.001,type=float,help='')
    argparser.add_argument('--syn_twoStage',default=True,type=bool,help='')
    argparser.add_argument('--lrate',default=0.001,type=float,help='learning rate')
    # About Debug or Show
    argparser.add_argument('--verbose',default=1,type=int,help='The level of verbose')
    argparser.add_argument('--epoch_show',default=5,type=int,help='The epochs of show time')
    # About Regression_t
    argparser.add_argument('--regt_batch_size',default=500,type=int,help='The size of one batch')
    argparser.add_argument('--regt_lr',default=0.05,type=float,help='The learning rate')
    argparser.add_argument('--regt_num_epoch',default=5,type=int,help='The num of total epoch')
    # About IRM  
    argparser.add_argument('--env_list',default=[-3.0, 3.0],type=list,help='The environment list')
    argparser.add_argument('--env_str',default='[1.3, 3.0]',type=str,help='The environment list')
    argparser.add_argument('--data_dict',default={},type=dict,help='The data dict')
    # args = argparser.parse_args()
    args = argparser.parse_args(args=[])
    return args

args = get_args()

# OOD结果画图

In [9]:
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# set OOD path
''' bias rate 1'''
br = [-3.0, -2.5, -2.0, -1.5, -1.3, 1.3, 1.5, 2.0, 2.5, 3.0, 0.0]
brdc = {-3.0: 'n30', -2.5:'n25', -2.0:'n20', -1.5:'n15', -1.3:'n13', 1.3:'p13', 1.5:'p15', 2.0:'p20', 2.5:'p25', 3.0:'p30', 0.0:'0'}
which_benchmark = 'SynOOD2_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one, args.depX, args.depU,args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU, args.mXs])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}_{args.mode}/ood{brdc[args.ood]}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
# os.makedirs(os.path.dirname(resultDir), exist_ok=True)
logfile = f'{resultDir}/log.txt'

In [7]:
resultDir

'../../Data//results/SynOOD2_1_0_1_0.05_0.05_1_2_10_4_2_vx/ood0/'

In [13]:
import numpy as np

% matplotlib notebook
# 从CSV文件加载数据
''' bias rate '''
br = [-3.0, -2.5, -2.0, -1.5, -1.3, 0.0, 1.3, 1.5, 2.0, 2.5, 3.0]
brdc = {-3.0: 'n30', -2.5:'n25', -2.0:'n20', -1.5:'n15', -1.3:'n13', 1.3:'p13', 1.5:'p15', 2.0:'p20', 2.5:'p25', 3.0:'p30', 0.0:'0'}
data_ate = pd.read_csv(f'{resultDir}/CBIV_vx_results_ood_ate_cbiv.csv')
data_pehe = pd.read_csv(f'{resultDir}/CBIV_vx_results_ood_pehe_cbiv.csv')
new_order = ['n30','n25','n20','n15','n13','0','p13','p15','p20','p25','p30']
data_ate = data_ate.reindex(columns=new_order)
data_pehe = data_pehe.reindex(columns=new_order)

def custom_color(x, iid):
    x = np.array(x)
    colors = np.where(x == iid, 'blue', 'red')
    return colors

# 计算每一列的均值 方差
mean_values_ate = np.mean(data_ate[:][:10], axis=0)
std_values_ate = np.std(data_ate[:][:10], axis=0)
mean_values_pehe = np.mean(data_pehe[:][:10], axis=0)
std_values_pehe = np.std(data_pehe[:][:10], axis=0)

fig, (ax1, ax2) = plt.subplots(1,2, figsize = (10,4), dpi = 200)
ax1.grid(ls = "--", lw = 0.5, color = "#4E616C")
ax1.errorbar(br, mean_values_ate, yerr=std_values_ate, marker='o', label='PEHE')
ax1.set_xlabel('bias rate')
ax1.set_ylabel('ATE')
ax2.grid(ls = "--", lw = 0.5, color = "#4E616C")
ax2.errorbar(br, mean_values_pehe, yerr=std_values_pehe, marker='o', label='PEHE')
ax2.set_xlabel('bias rate')
ax2.set_ylabel('PEHE')
fig.suptitle(f"Syn-2-4-4-2 OOD:{args.ood}")
plt.savefig(f'{resultDir}/CBIV_vx_ood{brdc[args.ood]}.png')
plt.legend()
plt.show()


<IPython.core.display.Javascript object>

## 汇总图

In [28]:
import numpy as np
import matplotlib.pyplot as plt
# l = [-3.0, -2.5,-2.0, 0.0, 2.0, 2.5, 3.0]
# p10,p13,p15,p20,p25,p30,p35,p40,p45,p50
l = [1.0, 1.3,4.5, 5.0]
% matplotlib notebook
mean_ate_list = []
std_ate_list = []
mean_pehe_list = []
std_pehe_list = []
for r in l:
    args.ood = r
    which_benchmark = 'SynOOD_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one, args.depX, args.depU,args.VX])
    which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU, args.mXs])
    resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}_{args.mode}/ood{brdc[args.ood]}/'
    
    data_ate = pd.read_csv(f'{resultDir}/CBIV_{args.mode}_results_ood_ate_cbiv_positive.csv')
    data_pehe = pd.read_csv(f'{resultDir}/CBIV_{args.mode}_results_ood_pehe_cbiv_positive.csv')
    # new_order = ['n30','n25','n20','n15','n13','0','p13','p15','p20','p25','p30']
    # data_ate = data_ate.reindex(columns=new_order)
    # data_pehe = data_pehe.reindex(columns=new_order)

    # 计算每一列的均值 方差
    mean_values_ate = np.mean(data_ate[:][:20], axis=0)
    std_values_ate = np.std(data_ate[:][:20], axis=0)
    mean_values_pehe = np.mean(data_pehe[:][:20], axis=0)
    std_values_pehe = np.std(data_pehe[:][:20], axis=0)

    mean_ate_list.append(mean_values_ate)
    std_ate_list.append(std_values_ate)
    mean_pehe_list.append(mean_values_pehe)
    std_pehe_list.append(std_values_pehe)


fig, (ax1, ax2) = plt.subplots(1,2, figsize = (10,4), dpi = 200)
ax1.grid(ls = "--", lw = 0.5, color = "#4E616C")
ax1.errorbar(br, mean_ate_list[0], yerr=std_ate_list[0], marker='o', label='1.0')
ax1.errorbar(br, mean_ate_list[1], yerr=std_ate_list[1], marker='v', label='1.3')
ax1.errorbar(br, mean_ate_list[2], yerr=std_ate_list[2], marker='^', label='4.5')
ax1.errorbar(br, mean_ate_list[3], yerr=std_ate_list[3], marker='<', label='5.0')

# ax1.errorbar(br, mean_ate_list[0], yerr=std_ate_list[0], marker='o', label='-3.0')
# ax1.errorbar(br, mean_ate_list[1], yerr=std_ate_list[1], marker='v', label='-2.5')
# ax1.errorbar(br, mean_ate_list[2], yerr=std_ate_list[2], marker='^', label='-2.0')
# ax1.errorbar(br, mean_ate_list[3], yerr=std_ate_list[3], marker='<', label='0.0')
# ax1.errorbar(br, mean_ate_list[4], yerr=std_ate_list[4], marker='>', label='2.0')
# ax1.errorbar(br, mean_ate_list[5], yerr=std_ate_list[5], marker='1', label='2.5')
# ax1.errorbar(br, mean_ate_list[6], yerr=std_ate_list[6], marker='2', label='3.0')
ax1.set_xlabel('bias rate')
ax1.set_ylabel('ATE')
ax2.grid(ls = "--", lw = 0.5, color = "#4E616C")
ax2.errorbar(br, mean_pehe_list[0], yerr=std_pehe_list[0], marker='o', label='1.0')
ax2.errorbar(br, mean_pehe_list[1], yerr=std_pehe_list[1], marker='v', label='1.3')
ax2.errorbar(br, mean_pehe_list[2], yerr=std_pehe_list[2], marker='^', label='4.5')
ax2.errorbar(br, mean_pehe_list[3], yerr=std_pehe_list[3], marker='<', label='5.0')

# ax2.errorbar(br, mean_pehe_list[0], yerr=std_pehe_list[0], marker='o', label='-3.0')
# ax2.errorbar(br, mean_pehe_list[1], yerr=std_pehe_list[1], marker='v', label='-2.5')
# ax2.errorbar(br, mean_pehe_list[2], yerr=std_pehe_list[2], marker='^', label='-2.0')
# ax2.errorbar(br, mean_pehe_list[3], yerr=std_pehe_list[3], marker='<', label='0.0')
# ax2.errorbar(br, mean_pehe_list[4], yerr=std_pehe_list[4], marker='>', label='2.0')
# ax2.errorbar(br, mean_pehe_list[5], yerr=std_pehe_list[5], marker='1', label='2.5')
# ax2.errorbar(br, mean_pehe_list[6], yerr=std_pehe_list[6], marker='2', label='3.0')
ax2.set_xlabel('bias rate')
ax2.set_ylabel('PEHE')
fig.suptitle(f"Syn-{args.mV}-{args.mX}-{args.mU}-{args.mXs}-{args.mode} OOD:total")
ax1.legend(loc='upper left')
ax2.legend(loc='upper left')
plt.savefig(f'{resultDir}/CBIV_vx_total_ood1.png')

plt.show()

<IPython.core.display.Javascript object>

# OOD-Aggr 画图

In [43]:
import ast
args.env_list = ast.literal_eval(args.env_str)
print(args.env_list)
# set path
''' bias rate 1'''
br = [-3.0, -2.5, -2.0, -1.5, -1.3, 1.3, 1.5, 2.0, 2.5, 3.0]
brdc = {-3.0: 'n30', -2.5:'n25', -2.0:'n20', -1.5:'n15', -1.3:'n13', 1.3:'p13', 1.5:'p15', 2.0:'p20', 2.5:'p25', 3.0:'p30', 0.0:'0'}

des_str = args.des_str
which_benchmark = 'SynOOD2_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one, args.depX, args.depU,args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU, args.mXs])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}_{args.mode}/'
for r in args.env_list:
    resultDir += f'{brdc[r]}_'
resultDir = resultDir[:-1]
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
resultDir

[1.3, 3.0]


'../../Data//results/SynOOD2_1_0_1_0.05_0.05_1_2_10_4_2_vx/p13_p30'

In [45]:
import numpy as np
import matplotlib.pyplot as plt
# l = [-3.0, -2.5,-2.0, 0.0, 2.0, 2.5, 3.0]
# p10,p13,p15,p20,p25,p30,p35,p40,p45,p50
l = [
    '1223_lr=0.001_reps=20_v1_split',
    '1223_lr=0.002_reps=20_v1_split',
    '1223_lr=0.003_reps=20_v1_split'
     ]
# l = [
#     '1220_lr=0.001_reps=20_v0',
#      '1220_lr=0.002_reps=20_v0'
#      ]
marker = ['o', 'v', '^', '<', '1', '2']
% matplotlib notebook
ate_str = 'CBIV_vx_results_ood_ate_val_obj.csv' 
pehe_str = 'CBIV_vx_results_ood_pehe_val_obj.csv'
mean_ate_list = []
std_ate_list = []
mean_pehe_list = []
std_pehe_list = []
for des in l:
    resultDir_temp = resultDir + '/' + des
    data_ate = pd.read_csv(f'{resultDir_temp}/{ate_str}')
    data_pehe = pd.read_csv(f'{resultDir_temp}/{pehe_str}')
    # new_order = ['n30','n25','n20','n15','n13','0','p13','p15','p20','p25','p30']
    # data_ate = data_ate.reindex(columns=new_order)
    # data_pehe = data_pehe.reindex(columns=new_order)
    print(resultDir_temp)
    # 计算每一列的均值 方差
    mean_values_ate = np.mean(data_ate[:][:20], axis=0)
    std_values_ate = np.std(data_ate[:][:20], axis=0)
    mean_values_pehe = np.mean(data_pehe[:][:20], axis=0)
    std_values_pehe = np.std(data_pehe[:][:20], axis=0)

    mean_values_ate = mean_values_ate.drop(['0'])
    std_values_ate = std_values_ate.drop(['0'])
    mean_values_pehe = mean_values_pehe.drop(['0'])
    std_values_pehe = std_values_pehe.drop(['0'])


    mean_ate_list.append(mean_values_ate)
    std_ate_list.append(std_values_ate)
    mean_pehe_list.append(mean_values_pehe)
    std_pehe_list.append(std_values_pehe)


fig, (ax1, ax2) = plt.subplots(1,2, figsize = (10,4), dpi = 200)
ax1.grid(ls = "--", lw = 0.5, color = "#4E616C")
ax1.set_xlabel('bias rate')
ax1.set_ylabel('ATE')
ax2.grid(ls = "--", lw = 0.5, color = "#4E616C")
ax2.set_xlabel('bias rate')
ax2.set_ylabel('PEHE')
for i in range(len(l)):
    ax1.errorbar(br, mean_ate_list[i], yerr=std_ate_list[i], marker=marker[i], label=l[i])
    ax2.errorbar(br, mean_pehe_list[i], yerr=std_pehe_list[i], marker=marker[i], label=l[i])

fig.suptitle(f"Syn-{args.mV}-{args.mX}-{args.mU}-{args.mXs}-{args.mode} OOD:aggr_p13p30_v1_split")
ax1.legend(loc='lower right')
ax2.legend(loc='lower right')
plt.savefig(f'{resultDir}/CBIV_aggr_p13p30_v1_split.png')

plt.show()
print(resultDir)
mean_values_ate

../../Data//results/SynOOD2_1_0_1_0.05_0.05_1_2_10_4_2_vx/p13_p30/1223_lr=0.001_reps=20_v1_split
../../Data//results/SynOOD2_1_0_1_0.05_0.05_1_2_10_4_2_vx/p13_p30/1223_lr=0.002_reps=20_v1_split
../../Data//results/SynOOD2_1_0_1_0.05_0.05_1_2_10_4_2_vx/p13_p30/1223_lr=0.003_reps=20_v1_split


<IPython.core.display.Javascript object>

../../Data//results/SynOOD2_1_0_1_0.05_0.05_1_2_10_4_2_vx/p13_p30


n30    0.944360
n25    0.945070
n20    0.948715
n15    0.956935
n13    0.973305
p13    1.188470
p15    1.188460
p20    1.190555
p25    1.189990
p30    1.190880
dtype: float64

# OOD结果画图

In [3]:
# set path
''' bias rate 1'''
br = [-3.0, -2.5, -2.0, -1.5, -1.3, 1.3, 1.5, 2.0, 2.5, 3.0]
brdc = {-3.0: 'n30', -2.5:'n25', -2.0:'n20', -1.5:'n15', -1.3:'n13', 1.3:'p13', 1.5:'p15', 2.0:'p20', 2.5:'p25', 3.0:'p30', 0.0:'0'}

args.ood = -3.0
which_benchmark = 'SynOOD2_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one, args.depX, args.depU,args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU, args.mXs])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}_{args.mode}/ood{brdc[args.ood]}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'

resultDir

'../../Data//results/SynOOD2_1_0_1_0.05_0.05_1_2_10_4_2_vx/oodn30/'

In [5]:
import numpy as np
import matplotlib.pyplot as plt
# l = [-3.0, -2.5,-2.0, 0.0, 2.0, 2.5, 3.0]
# p10,p13,p15,p20,p25,p30,p35,p40,p45,p50
l = [
    # '1207_iter=20000_lr=0.0015_reps=10_all',
    # '1208_iter=30000_lr=0.002_reps=20_all',
    '1219_lr=0.001_reps=20_v0',
    '1219_lr=0.002_reps=20_v0',
    '1212_iter=30000_lr=0.001_reps=20',
    '1212_iter=30000_lr=0.002_reps=20'
     ]
# l = [
#     '1220_lr=0.001_reps=20_v0',
#      '1220_lr=0.002_reps=20_v0'
#      ]
marker = ['o', 'v', '^', '<', '1', '2']
% matplotlib notebook
ate_str = 'CBIV_vx_results_ood_ate_val_obj.csv' 
pehe_str = 'CBIV_vx_results_ood_pehe_val_obj.csv'
mean_ate_list = []
std_ate_list = []
mean_pehe_list = []
std_pehe_list = []
for des in l:
    resultDir_temp = resultDir + '/' + des
    data_ate = pd.read_csv(f'{resultDir_temp}/{ate_str}')
    data_pehe = pd.read_csv(f'{resultDir_temp}/{pehe_str}')
    # new_order = ['n30','n25','n20','n15','n13','0','p13','p15','p20','p25','p30']
    # data_ate = data_ate.reindex(columns=new_order)
    # data_pehe = data_pehe.reindex(columns=new_order)
    print(resultDir_temp)
    # 计算每一列的均值 方差
    mean_values_ate = np.mean(data_ate[:][:20], axis=0)
    std_values_ate = np.std(data_ate[:][:20], axis=0)
    mean_values_pehe = np.mean(data_pehe[:][:20], axis=0)
    std_values_pehe = np.std(data_pehe[:][:20], axis=0)

    mean_values_ate = mean_values_ate.drop(['0'])
    std_values_ate = std_values_ate.drop(['0'])
    mean_values_pehe = mean_values_pehe.drop(['0'])
    std_values_pehe = std_values_pehe.drop(['0'])


    mean_ate_list.append(mean_values_ate)
    std_ate_list.append(std_values_ate)
    mean_pehe_list.append(mean_values_pehe)
    std_pehe_list.append(std_values_pehe)


fig, (ax1, ax2) = plt.subplots(1,2, figsize = (10,4), dpi = 200)
ax1.grid(ls = "--", lw = 0.5, color = "#4E616C")
ax1.set_xlabel('bias rate')
ax1.set_ylabel('ATE')
ax2.grid(ls = "--", lw = 0.5, color = "#4E616C")
ax2.set_xlabel('bias rate')
ax2.set_ylabel('PEHE')
for i in range(len(l)):
    ax1.errorbar(br, mean_ate_list[i], yerr=std_ate_list[i], marker=marker[i], label=l[i])
    ax2.errorbar(br, mean_pehe_list[i], yerr=std_pehe_list[i], marker=marker[i], label=l[i])

fig.suptitle(f"Syn-{args.mV}-{args.mX}-{args.mU}-{args.mXs}-{args.mode} OOD:aggr_n30_v0")
ax1.legend(loc='lower right',fontsize='x-small')
ax2.legend(loc='lower right',fontsize='x-small')
plt.savefig(f'{resultDir}/CBIV_aggr_n30_v0.png')

plt.show()
print(resultDir)
mean_values_ate

../../Data//results/SynOOD2_1_0_1_0.05_0.05_1_2_10_4_2_vx/oodn30//1219_lr=0.001_reps=20_v0
../../Data//results/SynOOD2_1_0_1_0.05_0.05_1_2_10_4_2_vx/oodn30//1219_lr=0.002_reps=20_v0
../../Data//results/SynOOD2_1_0_1_0.05_0.05_1_2_10_4_2_vx/oodn30//1212_iter=30000_lr=0.001_reps=20
../../Data//results/SynOOD2_1_0_1_0.05_0.05_1_2_10_4_2_vx/oodn30//1212_iter=30000_lr=0.002_reps=20


<IPython.core.display.Javascript object>

../../Data//results/SynOOD2_1_0_1_0.05_0.05_1_2_10_4_2_vx/oodn30/


n30    1.226535
n25    1.228565
n20    1.230895
n15    1.242155
n13    1.255865
p13    0.980170
p15    0.943705
p20    0.923125
p25    0.917640
p30    0.915890
dtype: float64